#I- Gathering data 
In the github repo we will import all the csv file extracted with SQL from the stackoverflow database

In [1]:
!git clone https://github.com/ArnaudFRANCOISE/StackOverflow_tags_prediction.git
!ls
#import drive.MyDrive.preprocess_tools as pts
import StackOverflow_tags_prediction.preprocess_tools as pts
print(pts)

Cloning into 'StackOverflow_tags_prediction'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 64 (delta 18), reused 4 (delta 2), pack-reused 0
Unpacking objects: 100% (64/64), done.
Checking out files: 100% (32/32), done.
drive  sample_data  StackOverflow_tags_prediction
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
<module 'StackOverflow_tags_prediction.preprocess_tools' from '/content/StackOverflow_tags_prediction/preprocess_tools.py'>


In [2]:
import os
import pandas as pd

from os import listdir
from os.path import isfile, join

mypath = "/content/StackOverflow_tags_prediction/QueryResults/QueryResults/"

filenames = [f for f in listdir(mypath) if isfile(join(mypath, f))]
#gather all csv into one dataframe
#print(len(filenames))
data = pd.read_csv(mypath+filenames[0])
nb_data = 100
data = data[:nb_data]

for filename in filenames[1:]:
  d = pd.read_csv(mypath+filename)
  data = pd.concat([data, d[:nb_data]], axis=0)
  #print(data.shape)

#eliminate duplicate
data = data.drop_duplicates(subset=['Body'])
data

,Id,Body,Title,Tags,CreationDate
0,11237948,<p>This question is on the OpenCV functions <c...,"findHomography, getPerspectiveTransform, & get...",<c++><opencv><affinetransform><homography>,2012-06-28 04:08:43
1,3967620,<p>I've been messing about with member-functio...,Member-function pointers and phantom classes,<c++><member-function-pointers><phantom-refere...,2010-10-19 10:58:48
2,3967661,<p>I am trying to install Qt in my CentOS syst...,error while loading shared libraries: libstdc+...,<c++><qt><std><qmake><libstdc++>,2010-10-19 11:03:01
3,27963226,<p>i am using ndk to compile c++ code on andro...,"the bool parameter is wrong,given ""false"" ,but...",<android><c++><android-ndk>,2015-01-15 12:15:16
4,27963230,"<p>According to the <a href=""http://www.iso.or...","Why Does the New C++ Standard use ""ISO/IEC 148...",<c++><c++14><iso>,2015-01-15 12:15:26
...,...,...,...,...,...
95,3415646,<p>I have a query that runs in an Excel Macro ...,MSQuery query changed in Excel,<excel><excel-2003><vba>,2010-08-05 14:22:55
96,45599064,<p>I want to my macro to send a message just w...,How I make my Worksheet_change react after the...,<excel><vba>,2017-08-09 19:24:16
97,45599114,"<p>Help, i want to update my table <em>PARKING...","access vba update one table, multiple fields",<sql><vba><ms-access>,2017-08-09 19:27:00
98,62857967,<p>I have been utilizing the <strong>Analysis ...,How to manually find residual/regression SS an...,<excel><vba><statistics>,2020-07-12 06:51:53


Concatenate Body and title column and drop the creationDate

In [3]:

#data = data.drop(['Id', 'CreationDate'], axis=1)
data.shape
data
X = data["Title"] + data["Body"]
X
data["Body"] = pd.concat([X, data["Tags"]], axis=1)
data = data.drop(["Title", "CreationDate"], axis=1)
data

,Id,Body,Tags
0,11237948,"findHomography, getPerspectiveTransform, & get...",<c++><opencv><affinetransform><homography>
1,3967620,Member-function pointers and phantom classes<p...,<c++><member-function-pointers><phantom-refere...
2,3967661,error while loading shared libraries: libstdc+...,<c++><qt><std><qmake><libstdc++>
3,27963226,"the bool parameter is wrong,given ""false"" ,but...",<android><c++><android-ndk>
4,27963230,"Why Does the New C++ Standard use ""ISO/IEC 148...",<c++><c++14><iso>
...,...,...,...
95,3415646,MSQuery query changed in Excel<p>I have a quer...,<excel><excel-2003><vba>
96,45599064,How I make my Worksheet_change react after the...,<excel><vba>
97,45599114,"access vba update one table, multiple fields<p...",<sql><vba><ms-access>
98,62857967,How to manually find residual/regression SS an...,<excel><vba><statistics>


## Target selection
select only the tags that we want

In [4]:
data["Tags"]
data["Tags"].iloc[0]
target_interest  = ["<c#>", "<c++>", "<java>", "<javascript>", "<html>","<css>","<android>", 
                    "<python>", "<r>", "<sql>"]
#target_interest = list(str_.lower() for str_ in target_interest)
print(len(target_interest))
print(target_interest)


10
['<c#>', '<c++>', '<java>', '<javascript>', '<html>', '<css>', '<android>', '<python>', '<r>', '<sql>']


In [5]:
#select only the target we are interested in
selected_target=[]
is_in = []

for tags in data["Tags"].iloc[:]:
  #print(tags)
  for tag in target_interest:
    is_in.append(tag in tags)
  
  if any(is_in) == True:
    #print(True)
    selected_target.append(1)
  else:
    #print(False)
    selected_target.append(0)

  is_in = []
  #for int_tags in target_interest:
print(selected_target)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [6]:
print(len(selected_target))
data["interest"] = selected_target
data

2558


,Id,Body,Tags,interest
0,11237948,"findHomography, getPerspectiveTransform, & get...",<c++><opencv><affinetransform><homography>,1
1,3967620,Member-function pointers and phantom classes<p...,<c++><member-function-pointers><phantom-refere...,1
2,3967661,error while loading shared libraries: libstdc+...,<c++><qt><std><qmake><libstdc++>,1
3,27963226,"the bool parameter is wrong,given ""false"" ,but...",<android><c++><android-ndk>,1
4,27963230,"Why Does the New C++ Standard use ""ISO/IEC 148...",<c++><c++14><iso>,1
...,...,...,...,...
95,3415646,MSQuery query changed in Excel<p>I have a quer...,<excel><excel-2003><vba>,0
96,45599064,How I make my Worksheet_change react after the...,<excel><vba>,0
97,45599114,"access vba update one table, multiple fields<p...",<sql><vba><ms-access>,1
98,62857967,How to manually find residual/regression SS an...,<excel><vba><statistics>,0


In [7]:
data =data[data.interest != 0]
data = data.drop("interest", axis = 1)
data

,Id,Body,Tags
0,11237948,"findHomography, getPerspectiveTransform, & get...",<c++><opencv><affinetransform><homography>
1,3967620,Member-function pointers and phantom classes<p...,<c++><member-function-pointers><phantom-refere...
2,3967661,error while loading shared libraries: libstdc+...,<c++><qt><std><qmake><libstdc++>
3,27963226,"the bool parameter is wrong,given ""false"" ,but...",<android><c++><android-ndk>
4,27963230,"Why Does the New C++ Standard use ""ISO/IEC 148...",<c++><c++14><iso>
...,...,...,...
41,3398072,python: is there a frequency function?<p>in ex...,<python><excel><vba>
56,54054315,VBA HTML Navigate Through Listings<p>I've got ...,<html><excel><vba><web-scraping>
91,3414023,iterate over database query results in vba in ...,<sql><excel><vba><cell>
97,45599114,"access vba update one table, multiple fields<p...",<sql><vba><ms-access>


#II - Clean all corpus


In [8]:
data['Body'] = data['Body'].apply(pts.preprocess) #<--- Really important line ! 18min execution

data['Body'].iloc[-1]

'error call vba function vsto codei follow vba code modul one xlt file public sub setshapetopbyv shapenam As string byval topvalu As singl thisworkbookactivesheetshapesshapenametop topvalu end sub In vsto addin I follow C code tri call vba function mworksheetapplicationrunsetshapetop kvkey kvvalu typemiss typemiss typemiss typemiss typemiss typemiss typemiss typemiss typemiss typemiss typemiss typemiss typemiss typemiss typemiss typemiss typemiss typemiss typemiss typemiss typemiss typemiss typemiss typemiss typemiss typemiss typemiss typemiss that line throw except follow excerpt log net log except comexcept target cluster report invok method macro execut target method cluster report paramet messag can run macro setshapetop the macro may avail workbook macro may disabl stack trace server stack trace except rethrown systemruntimeremotingproxiesrealproxyhandlereturnmessageimessag reqmsg imessag retmsg systemruntimeremotingproxiesrealproxyprivateinvokemessagedataamp msgdata int type micr

In [9]:
#save the datafram on drive
import pickle
pickle_out = open("/content/drive/MyDrive/Etudes/IA/stackoverflow_/dataframe_top14.pickle","wb")
pickle.dump(data, pickle_out)
pickle_out.close()